In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('/Users/ting2liu/data/kiva/kiva_ds_csv/NEW/loans.csv', parse_dates=["POSTED_TIME", "PLANNED_EXPIRATION_TIME",
                               "DISBURSE_TIME", "RAISED_TIME"])

In [3]:
NEW_names = ["LOAN_ID","LOAN_NAME","ORIGINAL_LANGUAGE","DESCRIPTION","DESCRIPTION_TRANSLATED","FUNDED_AMOUNT","LOAN_AMOUNT","STATUS","IMAGE_ID","VIDEO_ID","ACTIVITY_NAME","SECTOR_NAME","LOAN_USE","COUNTRY_CODE","COUNTRY_NAME","TOWN_NAME","CURRENCY_POLICY","CURRENCY_EXCHANGE_COVERAGE_RATE","CURRENCY","PARTNER_ID","POSTED_TIME","PLANNED_EXPIRATION_TIME","DISBURSE_TIME","RAISED_TIME","LENDER_TERM","NUM_LENDERS_TOTAL","NUM_JOURNAL_ENTRIES","NUM_BULK_ENTRIES","TAGS","BORROWER_NAMES","BORROWER_GENDERS","BORROWER_PICTURED","REPAYMENT_INTERVAL","DISTRIBUTION_MODEL"]

In [4]:
OLD_names = ['id','name','original_language','original_description','translated_description','funded_amount','loan_amount','status','image_id','video_id','activity','sector','use','country_code','country_name','town','currency_policy', 'currency_exchange_coverage_rate','currency','partner_id','posted_time','planned_expiration_time','disbursed_time','funded_time','term_in_months','lender_count','journal_entries_count','bulk_journal_entries_count','tags','borrower_names','borrower_genders','borrower_pictured','repayment_interval','distribution_model']

In [5]:
dic = {}
for (k,v) in zip(NEW_names, OLD_names):
    dic[k]=v

In [6]:
df = df.rename(columns=dic)

In [8]:
df.head().transpose()

,0,1,2,3,4
id,657307,657259,658010,659347,656933
name,Aivy,Idalia Marizza,Aasia,Gulmira,Ricky\t
original_language,English,Spanish,English,Russian,English
original_description,"Aivy, 21 years of age, is single and lives in ...","Doña Idalia, esta casada, tiene 57 años de eda...",Aasia is a 45-year-old married lady and she ha...,"Гулмире 36 лет, замужем, вместе с супругом вос...",Ricky is a farmer who currently cultivates his...
translated_description,NaN,"Idalia, 57, is married and lives with her husb...",NaN,Gulmira is 36 years old and married. She and ...,NaN
funded_amount,125,400,400,625,425
loan_amount,125,400,400,625,425
status,funded,funded,funded,funded,funded
image_id,1.51125e+06,1.50795e+06,1.51224e+06,1.51442e+06,1.50683e+06
video_id,NaN,NaN,NaN,NaN,NaN


### Create Region Column

In [10]:
region = pd.read_csv("~/data/kiva/kiva_ds_csv/region.csv")
dic = {k:v for (k,v) in zip(region.Name, region.Region)}
df["region"] = df.country_name.map(lambda x:dic[x])

### Create Year Column

In [11]:
df['year'] = df['posted_time'].dt.year #create year column

In [12]:
df.shape

(1419607, 36)

In [13]:
df = df[(df['status'] == 'funded') | (df['status'] == 'expired')]  #only take rows with 'funded' or 'expired' status

In [14]:
df.shape

(1409421, 36)

In [16]:
df = df.query('year != 2018 and year !=2006')

In [17]:
df.shape

(1406529, 36)

In [18]:
df.year.value_counts()

2017    220806
2016    196902
2015    181105
2014    173815
2013    139700
2012    133519
2011    112211
2010     91275
2009     80820
2008     50341
2007     26035
Name: year, dtype: int64

In [19]:
df['time_to_funding'] = df.funded_time - df.posted_time

In [20]:
df['hours_to_funding'] = df.time_to_funding.astype('timedelta64[h]')

In [21]:
df = df.query('hours_to_funding<2161 or hours_to_funding != hours_to_funding')

In [22]:
df.shape

(1404481, 38)

In [24]:
df = df.query('posted_time>=2012')

In [25]:
df.shape

(1044349, 38)

In [31]:
df.dropna(subset=['borrower_genders'], inplace=True)  # not interested in rows where gender is missing
df.reset_index(drop=True,inplace=True)

In [32]:
df.shape

(1018065, 38)

In [33]:
def genders_groups(x):
    x = x.split(', ')
    if len(x) == 1:
        return x[0]
    else:
        genders = Counter(x).keys()
        if 'female' in genders and 'male' not in genders:
            return 'female_group'
        elif 'female' in genders and 'male' in genders:
            return 'mixed_group'
        else:
            return 'male_group'

In [34]:
df['gender_group'] = df['borrower_genders'].map(genders_groups)

In [35]:
df.shape

(1018065, 39)

In [36]:
pd.DataFrame({'NaNs': df.isnull().sum()})

,NaNs
id,0
name,2891
original_language,0
original_description,13
translated_description,170992
funded_amount,0
loan_amount,0
status,0
image_id,0
video_id,1017730


In [37]:
df.head().transpose()

,0,1,2,3,4
id,657307,657259,658010,659347,656933
name,Aivy,Idalia Marizza,Aasia,Gulmira,Ricky\t
original_language,English,Spanish,English,Russian,English
original_description,"Aivy, 21 years of age, is single and lives in ...","Doña Idalia, esta casada, tiene 57 años de eda...",Aasia is a 45-year-old married lady and she ha...,"Гулмире 36 лет, замужем, вместе с супругом вос...",Ricky is a farmer who currently cultivates his...
translated_description,NaN,"Idalia, 57, is married and lives with her husb...",NaN,Gulmira is 36 years old and married. She and ...,NaN
funded_amount,125,400,400,625,425
loan_amount,125,400,400,625,425
status,funded,funded,funded,funded,funded
image_id,1.51125e+06,1.50795e+06,1.51224e+06,1.51442e+06,1.50683e+06
video_id,NaN,NaN,NaN,NaN,NaN


In [38]:
data = df[['gender_group', 'loan_amount', 'status', 'activity', 'sector', 'country_name', 'year', 'term_in_months', 'region']]  # cols

In [39]:
data.shape

(1018065, 9)

In [40]:
data.to_csv('loans_FINAL.csv', index=False)